In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col,collect_list
from pyspark.sql.types import StringType, ArrayType, DoubleType,IntegerType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RankingEvaluator
from HR import HitRate

from pyspark.sql.types import *
from pyspark.sql import functions as F

#For windows user only
import os 
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Create spark session

In [ ]:
spark = SparkSession.builder.master("local[*]") \
                    .config('spark.ui.showConsoleProgress', 'false')\
                    .appName('MovieRecomender') \
                    .getOrCreate()
                    
spark.sparkContext.setCheckpointDir('checkpoint/')

In [ ]:
schema =             StructType([
                    StructField('UserID', LongType(), True),
                     StructField('MovieID', LongType(), True),
                     StructField('Rating', IntegerType(), True),
                     StructField('Timestamp', LongType(), True),
                     ])

In [ ]:
df = spark.read.option("sep", "::").schema(schema).csv("data/ml-1m/ratings.dat")
df = df.toDF(*["UserID", "MovieID", "Rating", "Timestamp"])
df.createOrReplaceTempView("dataset");
df = df.dropna()
df.cache().count() #Force cache

# Model config

In [ ]:
als = ALS(
    rank=10,
    maxIter=15,
    regParam=0.05,
    
    userCol="UserID",
    itemCol="MovieID",
    ratingCol="Rating",
    implicitPrefs=False,
    coldStartStrategy='drop',
    nonnegative=False,
)

In [ ]:
rankRange = [10, 15]
iterRange = [10, 15]
regParamRange = [0.05, 0.1]

# Hit Rate

In [ ]:
def evaluate_hit_rate(als, left_out_df, keep_one_df, full_matrix, n_users):
    hr_evaluator = HitRate(predictionCol='prediction', labelCol='Rating', userCol='UserID', itemCol = "MovieID")
    value = hr_evaluator.eval(als, left_out_df, keep_one_df, full_matrix, n_users)
    return value

# RMSE

In [ ]:
def evaluate_rmse(model, test):
    rmse = RegressionEvaluator(metricName="rmse", labelCol="Rating", predictionCol="prediction")       
    predictions=model.transform(test).na.drop()
    return rmse.evaluate(predictions)

# NCDG at K

In [ ]:
def getRank(a):
    ret=[]
    for i in a:
        ret.append(float(i.MovieID))
    return ret
def toDouble(a):
    return [float(i) for i in a]

convertUDF = udf(lambda z: getRank(z),ArrayType(DoubleType()))
toDoubleUDF = udf(lambda z: toDouble(z),ArrayType(DoubleType()))

In [ ]:
def evaluate_ndcg_at_k(model, df, n_items, k = 10):
    ndcg = RankingEvaluator(labelCol="RealRank", predictionCol="recommendations",metricName="ndcgAtK", k=k)
    tempt=df.sort(col('Rating').desc()).groupBy("UserID").agg(collect_list('MovieID').alias("RealRank"))
    tempt=tempt.withColumn("RealRank",toDoubleUDF(col("RealRank")))

    rec=model.recommendForAllUsers(n_items).join(tempt,"UserID","inner")
    rec=rec.withColumn("recommendations",convertUDF(col("recommendations")))
    return ndcg.evaluate(rec)

# Params testing

In [ ]:
#Leave one out for calculating hit rate
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

windowSpec  = Window.partitionBy("UserID").orderBy(F.col("Rating").desc())
tmp = df.withColumn("row_number", row_number().over(windowSpec))      
left_out_dataframe = tmp.filter(F.col("row_number") != 1)
keep_one_dataframe = tmp.filter(F.col("row_number") == 1)
left_out_count = left_out_dataframe.persist().count() #Force persist due to size
keep_out_count = keep_one_dataframe.persist().count() #Force persist
print("Left out {}, training to evaluate hit rate on {}.".format(keep_out_count, left_out_count))

In [ ]:
user_df = df.select("UserID").distinct()
movie_df = df.select("MovieID").distinct()

user_df.cache().count() #Force cache
movie_df.cache().count() #Force cache
full_matrix = user_df.crossJoin(movie_df)
full_matrix.persist().count() #Force persist due to size

#Count
n_users = user_df.count()
n_items = movie_df.count()
for rank in rankRange:
    for iter in iterRange:
        for regParam in regParamRange:
            als = ALS(
                rank=rank,
                maxIter=iter,
                regParam=regParam,
                
                userCol="UserID",
                itemCol="MovieID",
                ratingCol="Rating",
                implicitPrefs=False,
                coldStartStrategy='drop',
                nonnegative=False,
            )
            hit_rate = (evaluate_hit_rate(als, left_out_dataframe, keep_one_dataframe, \
                                     full_matrix, n_users))
            
            # NDCG and RMSE
            
            (train, test) = df.randomSplit([0.8, 0.2])  
            model = als.fit(train)
            ndcg = (evaluate_ndcg_at_k(model, df, n_items))
            rmse = (evaluate_rmse(model, test))
            
            print("Evaluating, rank: {}, iter: {}, regParam: {}".format(rank, iter, regParam))
            print("NDCG: {}, RMSE: {}, Hit: {}".format(ndcg, rmse, hit_rate))